In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from pandas_datareader import DataReader
from pandas import Series, DataFrame

In [2]:
filedate = '20220315'

In [3]:
df_stock = pd.read_csv('data/stock_' + filedate + '.csv', index_col=0, header=[0,1], parse_dates=[0])

In [4]:
# datetimeだけだと年またぎの計算はできない！
# difdate = datetime(end.year, end.month, end.day -20)
# その場合は、timedelta使用する
# 今日から20日前の日付を取得※余裕をもたせてる
end = dt.datetime.now()
difdate = end - dt.timedelta(days = 30)

In [5]:
# 銘柄ごとの上位3番の金額を取得する＜最安値＞
def getmax(series, topnum=3, getmin=True, getindex=False):
  if getindex is False:
    series = series.sort_values(ascending=getmin).head(topnum).reset_index(drop=True)
    series.index += 1
    return series
  else:
    return series.sort_values(ascending=getmin).head(topnum).index

df_min_price1 = df_stock.apply(getmax, axis=0, topnum=3, getindex = False)

# 'Low'と'Adj Close'だけ残して他は削除 , 'Close', 'Open', 'Volume'),level=1, inplace=True
df_min_price1.drop(columns=['Close', 'Open', 'High', 'Volume'], inplace=True)

# indexの順番を入れ替えて、ソート
df_min_price1 = df_min_price1.swaplevel('Attributes','Symbols', axis=1)
df_min_price1.sort_values(by=['Symbols', 'Attributes'], axis=1, inplace=True)

# 終値は最安値を抽出
df_aclose = df_min_price1.loc[1, (slice(None), 'Adj Close')]

# SeriesをDataFrameに変換。その際にMaltiIndexのSymbolsをIndexに設定
df_aclose = df_aclose.unstack(level=1)

# 安値は2番めに安い金額を抽出（終値が最大の場合、その日の安値も最大であるため）
df_low = df_min_price1.loc[2, (slice(None), 'Low')]
df_low = df_low.unstack(level=1)

# DataFrameをIndexでマージ
df_min_price2 = pd.merge(df_aclose, df_low, left_index=True, right_index=True)

# 銘柄ごとの上位3番の日付を取得する＜最安値＞
def getmax(series, topnum=3, getmin=True, getindex=False):
  if getindex is False:
    series = series.sort_values(ascending=getmin).head(topnum).reset_index(drop=True)
    series.index += 1
    return series
  else:
    return series.sort_values(ascending=getmin).head(topnum).index

df_min_date1 = df_stock.apply(getmax, axis=0, topnum=3, getindex = True)

# 'High'と'Adj Close'だけ残して他は削除 , 'Close', 'Open', 'Volume'),level=1, inplace=True
df_min_date1.drop(columns=['Close', 'Open', 'High', 'Volume'], inplace=True)

# indexの順番を入れ替えて、ソート
df_min_date1 = df_min_date1.swaplevel('Attributes','Symbols', axis=1)
df_min_date1.sort_values(by=['Symbols', 'Attributes'], axis=1, inplace=True)

# 終値は最高値を出した日を抽出
df_aclose_d = df_min_date1.loc[0, (slice(None), 'Adj Close')]

# SeriesをDataFrameに変換。その際にMaltiIndexのSymbolsをIndexに設定
df_aclose_d = df_aclose_d.unstack(level=1)

# 安値は2番めに安い金額を出した日を抽出（終値が最小の場合、その日の安値も最小であるため）
df_low_d = df_min_date1.loc[1, (slice(None), 'Low')]
df_low_d = df_low_d.unstack(level=1)

# DataFrameをIndexでマージ
df_min_date2 = pd.merge(df_aclose_d, df_low_d, left_index=True, right_index=True)

# 列名を置換
df_min_date2.rename(columns = {'Adj Close':'Adjdate', 'Low':'Lowdate'}, inplace=True)

df_min_analysis = pd.merge(df_min_price2, df_min_date2, left_index=True, right_index=True)

# 日付の比較Datetimeから日付のみを取得して比較
# todayadj = end - dt.timedelta(days = 4)
df_min_analysis =df_min_analysis[(df_min_analysis['Adjdate'].dt.date == pd.to_datetime(filedate).date()) &
                 (df_min_analysis['Adj Close'] - df_min_analysis['Low'] < 0) &
                (abs(df_min_analysis['Adjdate'] - df_min_analysis['Lowdate']) >= dt.timedelta(days=3))
                 ]

df_min_analysis.to_csv('data/tsplus_min_' + filedate + '.csv')

In [6]:
# 銘柄ごとの上位3番の金額を取得する＜最高値＞
def getmax(series, topnum=3, getmin=False, getindex=False):
  if getindex is False:
    series = series.sort_values(ascending=getmin).head(topnum).reset_index(drop=True)
    series.index += 1
    return series
  else:
    return series.sort_values(ascending=getmin).head(topnum).index

df_max_price1 = df_stock.apply(getmax, axis=0, topnum=3, getindex = False)

# 'High'と'Adj Close'だけ残して他は削除 , 'Close', 'Open', 'Volume'),level=1, inplace=True
df_max_price1.drop(columns=['Close', 'Open', 'Low', 'Volume'], inplace=True)

# indexの順番を入れ替えて、ソート
df_max_price1 = df_max_price1.swaplevel('Attributes','Symbols', axis=1)
df_max_price1.sort_values(by=['Symbols', 'Attributes'], axis=1, inplace=True)

# 終値は最高値を抽出
df_aclose = df_max_price1.loc[1, (slice(None), 'Adj Close')]

# SeriesをDataFrameに変換。その際にMaltiIndexのSymbolsをIndexに設定
df_aclose = df_aclose.unstack(level=1)

# 高値は2番めに高い金額を抽出（終値が最大の場合、その日の高値も最大であるため）
df_high = df_max_price1.loc[2, (slice(None), 'High')]
df_high = df_high.unstack(level=1)

# DataFrameをIndexでマージ
df_max_price2 = pd.merge(df_aclose, df_high, left_index=True, right_index=True)

# 銘柄ごとの上位3番の日付を取得する＜最高値＞
def getmax(series, topnum=3, getmin=False, getindex=False):
  if getindex is False:
    series = series.sort_values(ascending=getmin).head(topnum).reset_index(drop=True)
    series.index += 1
    return series
  else:
    return series.sort_values(ascending=getmin).head(topnum).index

df_max_date1 = df_stock.apply(getmax, axis=0, topnum=3, getindex = True)

# 'High'と'Adj Close'だけ残して他は削除 , 'Close', 'Open', 'Volume'),level=1, inplace=True
df_max_date1.drop(columns=['Close', 'Open', 'Low', 'Volume'], inplace=True)

# indexの順番を入れ替えて、ソート
df_max_date1 = df_max_date1.swaplevel('Attributes','Symbols', axis=1)
df_max_date1.sort_values(by=['Symbols', 'Attributes'], axis=1, inplace=True)

# 終値は最高値を出した日を抽出
df_aclose_d = df_max_date1.loc[0, (slice(None), 'Adj Close')]

# SeriesをDataFrameに変換。その際にMaltiIndexのSymbolsをIndexに設定
df_aclose_d = df_aclose_d.unstack(level=1)

# 高値は2番めに高い金額を出した日を抽出（終値が最大の場合、その日の高値も最大であるため）
df_high_d = df_max_date1.loc[1, (slice(None), 'High')]
df_high_d = df_high_d.unstack(level=1)

# DataFrameをIndexでマージ
df_max_date2 = pd.merge(df_aclose_d, df_high_d, left_index=True, right_index=True)

# 列名を置換
df_max_date2.rename(columns = {'Adj Close':'Adjdate', 'High':'Highdate'}, inplace=True)

df_max_analysis = pd.merge(df_max_price2, df_max_date2, left_index=True, right_index=True)

# 日付の比較Datetimeから日付のみを取得して比較
# todayadj = end - dt.timedelta(days = 4)
df_max_analysis =df_max_analysis[(df_max_analysis['Adjdate'].dt.date == pd.to_datetime(filedate).date()) &
                 (df_max_analysis['Adj Close'] - df_max_analysis['High'] > 0) &
                (abs(df_max_analysis['Adjdate'] - df_max_analysis['Highdate']) >= dt.timedelta(days=3))
                 ]

df_max_analysis.to_csv('data/tsplus_max_' + filedate + '.csv')